Ray hat große Probleme vernünftig auf meiner Maschine zu laufen. 
Nach viel Debugging treten immer neue Fehler auf. 
Zu Zeitintensiv, Rückkehr zu SB3

In [1]:
import ray
import ray.rllib.agents.ppo as ppo
from ray.rllib.policy.policy import PolicySpec
from ray import tune
import sys, os
sys.path.append("../..")
os.environ["PYTHONPATH"] = "../.."
os.environ["TEST_TMPDIR"] = "../../logs/RAY"
from thesis.utils.utils import setup_matrix_for_ray


ray.shutdown()
ray.init(ignore_reinit_error=True, include_dashboard=True)
setup_matrix_for_ray()

In [2]:
config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 1
config["num_workers"] = 0
config["framework"] = "torch"
config["env"] = "matrix"
config["train_batch_size"] = config["sgd_minibatch_size"] = 4
config["env_config"] = dict(
    max_seconds = 10*60, 
    fleetsize = 4, 
    max_fleetsize = 10,
    config_args = dict(
        withCollisions = True,
        reward_target = 1, 
        reward_distance = 0.05,
        reward_block = -0.5, 
        dispatchinginterval=120,
        routinginterval = 2,
        dispatcher_args=dict(distance=1)
    )
)
config["multiagent"]={
    "policies":{ "agv": PolicySpec(), },
    "policy_mapping_fn": lambda agent_id, episode, worker,**kwargs: "agv",    
}
trainer = ppo.PPOTrainer(config)

2022-08-17 06:54:43,041	WARNING ppo.py:386 -- `train_batch_size` (4) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 4.
2022-08-17 06:54:43,041	INFO ppo.py:414 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-08-17 06:54:43,041	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
c:\Users\Wegma\.conda\envs\thesis\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_28799_wsb6545q)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
2022-08-17 06:54:46,044	WARNING env.py:42 -- Skipping env checking for this experiment
2022-08-17 06:54:51,135	WARNING util.py:65 -- Install gputil for GPU system monitoring

In [3]:
e=trainer.workers.local_worker().env

In [5]:
for i in range(10000):
    result = trainer.train()